# Make technosphere matrix
This notebook is to make technosphere matrix from the vectors of technosphere exchanges generated in notebook 2_3

### 1. Create technosphere matrix A

In [1]:
import pickle
import collections
import scipy.sparse
import numpy as np

In [3]:
with open('../../Data/tech_vector/regionalized_dataset_list.pickle', 'rb') as i:
    regionalized_dataset_list = pickle.load(i)

In [4]:
get_ipython().magic('time tec_mat = scipy.sparse.hstack([d[1] for d in regionalized_dataset_list])')

CPU times: user 2.4 s, sys: 190 ms, total: 2.59 s
Wall time: 2.59 s


In [5]:
# type(tec_mat) #scipy.sparse.csc.csc_matrix
# tec_mat.shape #(335099, 335099)
sum(tec_mat.diagonal().astype(bool))#3066 - these diagonal elements: market activity -> loss; ord trans act -> use product from itself

3067

In [6]:
np.max(tec_mat.diagonal()) #0.587301587301587

0.587301587301587

In [7]:
m = tec_mat.multiply(tec_mat < 0)
print('number of values in tec_mat:',len(tec_mat.data))
print('numbe rof negative values in tec_mat:',len(m.data))

number of values in tec_mat: 92774262
numbe rof negative values in tec_mat: 13454676


### 2. Create diagonal matrix I 

In [9]:
with open('../../Data/lci_iot_imported/cutoff371_no_mg.pickle', 'rb') as i:
    datasets = pickle.load(i)
code_dict = {d['code']:d for d in datasets}
with open('../../Data/tech_vector/full_index.p', 'rb') as i: # [(country, mr_key),()...]
    full_index = pickle.load(i)
with open('../../Data/tech_vector/full_reference_dataset_dict.p', 'rb') as i: #{(country, mr_key): reference dataset}
    reference_dataset_dict = pickle.load(i)

In [10]:
production_amount_dict = {d['code']:d['production amount'] for d in datasets}
collections.Counter(production_amount_dict.values())

Counter({1.0: 16739, -1.0: 2274})

In [11]:
get_ipython().magic('time production_vector = [production_amount_dict[reference_dataset_dict[i]] for i in full_index]')
production_diags = scipy.sparse.diags(production_vector, format='coo')
print(len(production_vector),production_diags.shape)

CPU times: user 172 ms, sys: 7.86 ms, total: 180 ms
Wall time: 178 ms
335099 (335099, 335099)


In [10]:
with open('../../Data/final_matrix/production_vector.p', 'wb') as o:
    pickle.dump(production_vector, o)

### 3. (A-I) used for future LCI calculation

In [12]:
get_ipython().magic('time tec_mat_abs = np.absolute(tec_mat)')
get_ipython().magic('time production_diags_abs = np.absolute(production_diags)')
get_ipython().magic('time technosphere_matrix_abs = scipy.sparse.csc_matrix(production_diags_abs - tec_mat_abs.tocoo())')
with open('../../Data/final_matrix/technosphere_matrix.p', 'wb') as o:
    pickle.dump(technosphere_matrix_abs, o)

CPU times: user 144 ms, sys: 240 ms, total: 384 ms
Wall time: 381 ms
CPU times: user 2.05 ms, sys: 665 µs, total: 2.72 ms
Wall time: 2.29 ms
CPU times: user 5.35 s, sys: 1.03 s, total: 6.38 s
Wall time: 6.39 s


In [13]:
technosphere_matrix_abs

<335099x335099 sparse matrix of type '<class 'numpy.float64'>'
	with 93106294 stored elements in Compressed Sparse Column format>

In [12]:
with open('../../Data/final_matrix/full_index.p', 'wb') as o:
    pickle.dump(full_index, o)